# Acquire

**Goal: Your goal is to predict the values of single unit properties using the obervations from 2017.**

**import**

In [57]:
# data manipulation
import pandas as pd
import numpy as np

# data visualization
import matplotlib.pyplot as plt
import seaborn as sns

# data separation/transformation
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

# system manipulation
import sys
sys.path.append("./util_")
import acquire_
import prepare_

# other
import warnings
warnings.filterwarnings("ignore")


**get data**

In [58]:
# sql query
query = """
SELECT bedroomcnt, 
        bathroomcnt,
        calculatedfinishedsquarefeet,
        taxvaluedollarcnt,
        yearbuilt,
        taxamount,
        fips     
FROM properties_2017
WHERE propertylandusetypeid = 261; -- Single family home
"""

In [59]:
# get data from codeup database
zillow, q = acquire_.get_codeup_sql_data_(db_name="zillow", query=query,fileName="zillow_single_family")


**Understand data**

In [60]:
zillow.head()

,bedroomcnt,bathroomcnt,calculatedfinishedsquarefeet,taxvaluedollarcnt,yearbuilt,taxamount,fips
0,0.0,0.0,NaN,27516.0,NaN,NaN,6037.0
1,0.0,0.0,NaN,10.0,NaN,NaN,6037.0
2,0.0,0.0,NaN,10.0,NaN,NaN,6037.0
3,0.0,0.0,NaN,2108.0,NaN,174.21,6037.0
4,4.0,2.0,3633.0,296425.0,2005.0,6941.39,6037.0


In [61]:
zillow.shape

(2152863, 7)

In [62]:
zillow.columns

Index(['bedroomcnt', 'bathroomcnt', 'calculatedfinishedsquarefeet',
       'taxvaluedollarcnt', 'yearbuilt', 'taxamount', 'fips'],
      dtype='object')

In [63]:
zillow.info(show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2152863 entries, 0 to 2152862
Data columns (total 7 columns):
 #   Column                        Non-Null Count    Dtype  
---  ------                        --------------    -----  
 0   bedroomcnt                    2152852 non-null  float64
 1   bathroomcnt                   2152852 non-null  float64
 2   calculatedfinishedsquarefeet  2144379 non-null  float64
 3   taxvaluedollarcnt             2152370 non-null  float64
 4   yearbuilt                     2143526 non-null  float64
 5   taxamount                     2148421 non-null  float64
 6   fips                          2152863 non-null  float64
dtypes: float64(7)
memory usage: 115.0 MB


In [64]:
# count all the null values
zillow.isna().sum().values.sum() / len(zillow)

0.01058032954256727

In [65]:
zillow.describe()

,bedroomcnt,bathroomcnt,calculatedfinishedsquarefeet,taxvaluedollarcnt,yearbuilt,taxamount,fips
count,2.152852e+06,2.152852e+06,2.144379e+06,2.152370e+06,2.143526e+06,2.148421e+06,2.152863e+06
mean,3.287196e+00,2.230688e+00,1.862855e+03,4.618962e+05,1.960950e+03,5.634866e+03,6.048377e+03
std,9.547544e-01,9.992796e-01,1.222125e+03,6.996760e+05,2.216220e+01,8.178910e+03,2.043329e+01
min,0.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00,1.801000e+03,1.850000e+00,6.037000e+03
25%,3.000000e+00,2.000000e+00,1.257000e+03,1.881702e+05,1.949000e+03,2.534980e+03,6.037000e+03
50%,3.000000e+00,2.000000e+00,1.623000e+03,3.276710e+05,1.958000e+03,4.108950e+03,6.037000e+03
75%,4.000000e+00,3.000000e+00,2.208000e+03,5.345270e+05,1.976000e+03,6.414320e+03,6.059000e+03
max,2.500000e+01,3.200000e+01,9.525760e+05,9.842891e+07,2.016000e+03,1.337756e+06,6.111000e+03


In [66]:
# count of numeric columns and object columns
numerics = len(zillow.select_dtypes("number").columns)
objects = len(zillow.select_dtypes("object").columns)

print("Numeric col count:", numerics)
print("object col count:", objects)

Numeric col count: 7
object col count: 0


**What I see:**

- I have 2152863 rows and 7 columns
- 7 of the 29 columns are numric while 0 of them are string object colums
- 22778 total null count (1% of the data)
- I also see the descriptive statistics of my data

# Prepare

**Rename columns**

In [67]:
# rename dataframe columns
zillow = zillow.rename(columns={"bedroomcnt":"bedrooms",
                       "bathroomcnt":"bathrooms",
                       "calculatedfinishedsquarefeet":"sqr_feet",
                      "taxvaluedollarcnt":"tax_value",
                      "yearbuilt":"year_built",
                      "taxamount":"tax_amount",
                      "fips":"county"})

**drop nulls**

In [68]:
# drop all nulls in the dataframe
zillow = zillow.dropna()

**convert data type**

In [69]:
# convert data type from float to int
zillow.bedrooms = zillow.bedrooms.astype(int)
zillow.year_built = zillow.year_built.astype(int)

**Remove duplicated rows**

In [70]:
# remove the duplocated rows
zillow = zillow.drop_duplicates(keep="first")

**Remove outliers**

This is done using box plot in the explore phase. only looking at the training data.

In [71]:
# remove outliers
zillow = zillow[zillow.bedrooms < 5]
zillow = zillow[zillow.bathrooms < 5]
zillow = zillow[(zillow.year_built >= 1910) & (zillow.year_built <= 2017)]
zillow = zillow[(zillow.sqr_feet > 500) & (zillow.sqr_feet <= 3800)]
zillow = zillow[zillow.tax_amount <= 20000]
zillow = zillow[zillow.tax_value <= 1_000_000]

In [76]:
zillow.shape

(1809680, 7)

**feature transformation**

In [77]:
# Rename the unique values in fips to county names
zillow.county = zillow.county.astype(str).str.replace("6037.0","Los Angeles").str.replace("6059.0","Orange").str.replace("6111.0","Ventura")


**Create dummies**

In [78]:
# create dummie variables
dummies = pd.get_dummies(zillow.county)

# clean dummie column names
dummies_col = dummies.columns.str.replace(" ", "_").str.lower()

dummies.head(2)

,Los Angeles,Orange,Ventura
4,1,0,0
6,1,0,0


In [79]:
# make a copy of my original data frame
zillow_encoded_scaled = zillow.copy()

# add dummies to my data frame
zillow_encoded_scaled[dummies_col] = dummies
zillow_encoded_scaled.head(2)

,bedrooms,bathrooms,sqr_feet,tax_value,year_built,tax_amount,county,los_angeles,orange,ventura
4,4,2.0,3633.0,296425.0,2005,6941.39,Los Angeles,1,0,0
6,3,4.0,1620.0,847770.0,2011,10244.94,Los Angeles,1,0,0


## Split

In [80]:
# split the data into training, validation and testing sets
train, validate, test = prepare_.split_data_(df=zillow_encoded_scaled,
                    test_size=0.2, 
                     validate_size=0.2,
                    stratify_col= "county",
                     random_state=95)
(train.shape, validate.shape, test.shape)

((1085808, 10), (361936, 10), (361936, 10))

**Scale data**

In [81]:
# scalable features
features_to_scale = train[['bedrooms','bathrooms','sqr_feet','year_built','tax_amount']]

features_to_scale.head(2)

,bedrooms,bathrooms,sqr_feet,year_built,tax_amount
1632095,3,2.0,1159.0,1955,4977.88
898614,2,1.0,968.0,1940,2780.27


In [82]:
# build a scaling object
scaler = MinMaxScaler()

# Note that we only call .fit with the training data,
# but we use .transform to apply the scaling to all the data splits.
# y Target values: this is not being changed in any way(no predictions are being made)
x_train_scaled = scaler.fit_transform(X=features_to_scale,)

# transfrom the validate and test using the minMax object
x_val_scaled = scaler.transform(X=validate[features_to_scale.columns])
x_test_scaled = scaler.transform(X=test[features_to_scale.columns])


In [83]:
x_train_scaled

array([[0.75      , 0.44444444, 0.19945438, 0.4245283 , 0.24870896],
       [0.5       , 0.22222222, 0.14155805, 0.28301887, 0.13846189],
       [0.75      , 0.44444444, 0.45710821, 0.4245283 , 0.2760609 ],
       ...,
       [1.        , 0.44444444, 0.48347984, 0.83018868, 0.21592294],
       [1.        , 0.44444444, 0.50136405, 0.16037736, 0.08784609],
       [0.        , 0.        , 0.07729615, 0.53773585, 0.10127323]])

**Convert scaled features to dataframe**

In [84]:
# New _ariable mames to add to data
new_scale_col = []
for i in features_to_scale.columns:
    new_scale_col.append(f"{i}_scaled")

new_scale_col

['bedrooms_scaled',
 'bathrooms_scaled',
 'sqr_feet_scaled',
 'year_built_scaled',
 'tax_amount_scaled']

In [85]:
# convert to dataframe
x_train_scaled = pd.DataFrame(x_train_scaled)
x_val_scaled = pd.DataFrame(x_val_scaled)
x_test_scaled = pd.DataFrame(x_test_scaled)

# add new column names back to the data frame
x_train_scaled[new_scale_col] = x_train_scaled
x_val_scaled[new_scale_col] = x_val_scaled
x_test_scaled[new_scale_col] = x_test_scaled

# remove redundent columns from new dataframe
x_train_scaled = x_train_scaled[new_scale_col]
x_val_scaled = x_val_scaled[new_scale_col]
x_test_scaled = x_test_scaled[new_scale_col]

x_train_scaled.head()

,bedrooms_scaled,bathrooms_scaled,sqr_feet_scaled,year_built_scaled,tax_amount_scaled
0,0.75,0.444444,0.199454,0.424528,0.248709
1,0.50,0.222222,0.141558,0.283019,0.138462
2,0.75,0.444444,0.457108,0.424528,0.276061
3,0.75,0.666667,0.526523,0.424528,0.268749
4,1.00,0.444444,0.239770,0.518868,0.158247


In [86]:
x_train_scaled.shape

(1085808, 5)

**Add scaled columns to a copy of original data**

In [87]:
# train[x_train_scaled.columns] = x_train_scaled
train = train.reset_index(drop=True)
validate = validate.reset_index(drop=True)
test = test.reset_index(drop=True)

# Concatenate the DataFrames horizontally
train = pd.concat([train, x_train_scaled], axis=1, verify_integrity=True)
validate = pd.concat([validate, x_val_scaled], axis=1, verify_integrity=True)
test = pd.concat([test, x_test_scaled], axis=1, verify_integrity=True)

# drop the original columns
train = train.drop(columns=features_to_scale)
validate = validate.drop(columns=features_to_scale)
test = test.drop(columns=features_to_scale)

In [88]:
train.head()

,tax_value,county,los_angeles,orange,ventura,bedrooms_scaled,bathrooms_scaled,sqr_feet_scaled,year_built_scaled,tax_amount_scaled
0,385794.0,Los Angeles,1,0,0,0.75,0.444444,0.199454,0.424528,0.248709
1,206832.0,Los Angeles,1,0,0,0.50,0.222222,0.141558,0.283019,0.138462
2,526078.0,Ventura,0,0,1,0.75,0.444444,0.457108,0.424528,0.276061
3,468240.0,Los Angeles,1,0,0,0.75,0.666667,0.526523,0.424528,0.268749
4,270136.0,Orange,0,1,0,1.00,0.444444,0.239770,0.518868,0.158247


**Save split**

In [89]:
# save created data frames into csv
prepare_.save_split_data_(original_df=zillow,
                          encoded_scaled_df=zillow_encoded_scaled, 
                          train=train, 
                          validate=validate, 
                          test=test,
                         test_size=0.2,
                         stratify_col= "county",
                         random_state=95)

'SIX data sets saved as .csv'

### Actions Taken

- Remove all nulls (1% of the data)
- Remove duplicated rows.
- convert data type from float to int (bedrooms, bathrooms)
- remove outliers
- replace the fips code with county names and Encode county column.
- Split data into train, validate, and test. (`60/20/20 ` split)
- scale the humeric categorical and continuous variable and extract a copy of the original data frame.
    - `bedrooms, bathrooms, sqr_feet, year_built, tax_amount`